<span style='color:#0066cc'> <span style='font-family:serif'> <font size="6"> **Access OSCAR Data with OPeNDAP**<span style='color:#0066cc'>

<font size="3.5"> Data sources:

- [OSCAR](https://podaac.jpl.nasa.gov/dataset/OSCAR_L4_OC_FINAL_V2.0) (Ocean Surface Current Analyses Real-time (OSCAR) Surface Currents - Final 0.25 Degree (Version 2.0)). 
- Iceberg Data (csv data from Kaggle)


<font size="3.5"> This simplified drifter model integrates ocean velocity (from OSCAR) forward in time.


In [ ]:
from pydap.net import create_session
from pydap.client import get_cmr_urls, consolidate_metadata, open_url
import xarray as xr
import datetime as dt
import json
import glob
import os
import numpy as np
import pandas as pd
import earthaccess
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from datetime import timedelta
from cftime import DatetimeJulian


<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Import Token Authorization and create Session**
 


<font size="3.5"> Here we use the Bearer Token to create an authenticated session. For this NASA-specific case, we will use earthaccess.



In [ ]:
auth = earthaccess.login(strategy="interactive")
fs = earthaccess.get_fsspec_https_session()
session_kwargs = {'token': fs.storage_options['client_kwargs']['headers']['Authorization'][7:]}

In [ ]:
my_session = create_session(use_cache=True, session_kwargs=session_kwargs)
my_session.cache.clear()

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Query opendap urls using NASA's CMR API**

In [ ]:
oscar_ccid = "C2098858642-POCLOUD" # https://podaac.jpl.nasa.gov/dataset/OSCAR_L4_OC_FINAL_V2.0

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Filter data via Temporal Searches**


In [ ]:
time_range = ['2019-08-16T00:00:00Z', '2020-09-16T00:00:00Z'] # 1 year of data
time_range

In [ ]:
ocean_urls = get_cmr_urls(ccid=oscar_ccid, time_range=time_range, session=my_session, limit=500)
print("found: ",len(ocean_urls), "OSCAR urls")
ocean_urls[-1]

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **OSCAR data**


In [ ]:
# Turn urls into DAP4 urls
opendap_OSCAR_urls = [url.replace("https", "dap4") for url in ocean_urls] # 

opendap_OSCAR_urls[:2]

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Consolidate metadata**

<font size="3.5"> All URLs belonging to the same Collection share many identical variables and metadata. The following function
reduces redundant metadata


In [ ]:
%%time
consolidate_metadata(opendap_OSCAR_urls, concat_dim='time', set_maps=True, session=my_session)

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Create Virtually Aggregated Dataset with Xarray**

<font size="3.5"> Now, you can create a virtually aggregated view of the dataset that is ready to analyze with Xarray and Pydap as an engine.

`ds_oscar` will contain all relevant ocean data.


In [ ]:
%%time
ds_oscar = xr.open_mfdataset(opendap_OSCAR_urls, engine='pydap', session=my_session, combine='nested', concat_dim="time", chunks={'latitude': 300})
ds_oscar

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Download only a subset of data and store locally**

<font size="3.5"> The dimensions `latitude`, `longitude` do not match the name of the coordinates, despite both being 1D arrays. However, to use these we can load the coordinates `lat` and `lon`  into memory and query the indexes longitude and latitude that match our coordinates of interest.

<font size="3.5"> We will need:
- <font size="3.5"> Load `lat`, `lon` into memory, using the xarray `.load()` method
- <font size="3.5"> Identify the range of `longitude` and `latitude` that match our subset, and the indexes associated with these.
- <font size="3.5"> Download only that data

In [ ]:
ds_oscar['lon'], ds_oscar['lat'] = ds_oscar['lon'].load(), ds_oscar['lat'].load()
ds_oscar = ds_oscar.rename_vars({'lon':'longitude', 'lat':'latitude'}).set_index(longitude='longitude').set_index(latitude='latitude').drop_vars(['ug', 'vg'])

### visualize subset

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10), subplot_kw=dict(projection=ccrs.PlateCarree()))

# Add filled land background
ax.add_feature(cfeature.LAND, facecolor='black', zorder=0)

cbar_kwargs={
        'shrink': 0.2,        # reduce height
        'aspect': 10,         # width-to-height ratio
        'pad': 0.05,          # spacing from the main plot
        'label': 'Zonal Velocity (m/s)'  # optional label
    }

# Plot ocean data on top
ds_oscar['u'].isel(time=0, latitude=slice(40, 300)).transpose().plot(ax=ax, transform=ccrs.PlateCarree(), cmap='hsv', cbar_kwargs=cbar_kwargs)
ax.coastlines()
plt.show()

## before downloding: re-chunk!

Rechunking instructs the Hyrax OPeNDAP server via xarray to download only those chunks at a time. This enables to construct a constraint expression via pydap, to request Hyrax  only that size of download 


In [ ]:
%%time
ds_oscar.isel(latitude=slice(40, 300)).to_netcdf("./data/Oscar_data.nc")

## Now that data is downloaded...

In [ ]:
path = "./data"
csv_files = sorted(glob.glob(os.path.join(path, "*.csv")))
dates = []
Coords = []
Times = []
DFs = []
for file in csv_files:
    df = pd.read_csv(file)
    df = df[~df['Remarks'].str.contains("grounded", na=False)]
    # DFs.append(df)
    times = len(df.index)*[dt.datetime.strptime(("-").join(df["Last Update"].values[0].split("/")), "%m-%d-%Y").strftime("%Y-%m-%dT%H:%M:%SZ")]
    Time = pd.DataFrame(times, columns=['time'], index=df.index)
    df['Date']=Time
df = df.drop(columns=['Remarks'])

In [ ]:
drifters = df
drifters.head(5)

In [ ]:
oscar_path = './data/Oscar_data.nc'

In [ ]:
ds_ovels = xr.open_dataset(oscar_path) # ocean velocities

In [ ]:
def forward_model(t, _Lats, _Lons, u, v):
    # sample all points from source data, using nearest neighbor (default)
    dt_seconds = 86400  # 1 day in seconds
    u0, v0 = u.sel(time=np.array(t)), v.sel(time=np.array(t))
    indexer = {
        "latitude": xr.DataArray(_Lats, dims="location"),
        "longitude": xr.DataArray(_Lons, dims="location"),
    }
    # Will assume these are in water, and nan -> 0 velocity
    nan_to_value = 0
    u_vals = u0.sel(**indexer, method='nearest').fillna(nan_to_value)
    v_vals = v0.sel(**indexer, method='nearest').fillna(nan_to_value)

    # Approximate degrees per meter
    lat_per_m = 1 / 111320
    lon_per_m = 1 / (111320 * np.cos(np.radians(_Lats)))

    # Update position due to velocity drift and wind-driven ekman
    # Approximation for large icebers with weak wind forcing

    nlat = v_vals * dt_seconds * lat_per_m
    nlon = u_vals * dt_seconds * lon_per_m
    
    nlat, nlon = nlat.rename('delta_lat'), nlon.rename('delta_lon')
    ncoords = pd.merge(nlat.to_dataframe(), nlon.to_dataframe()[['delta_lon']],left_index=True, right_index=True)

    # Use the input latitudes to update the output data
    ncoords['latitude'] = indexer['latitude'] +  ncoords['delta_lat']
    ncoords['longitude'] = indexer['longitude'] + ncoords['delta_lon']
    vels = pd.merge(u_vals.to_dataframe(), v_vals.to_dataframe())[['v', 'u']]
    # 
    return pd.concat([ncoords, vels], axis=1)[['time', 'latitude', 'longitude', 'v', 'u']]

In [ ]:
dt = timedelta(days=1)
results = []
# initial condition
time, Lats, Lons = drifters['Date'].values[0], drifters['Latitude'].values, drifters['Longitude'].values
# Parse into pandas Timestamp first
t_pd = pd.Timestamp(time)
t = DatetimeJulian(t_pd.year, t_pd.month, t_pd.day, t_pd.hour, t_pd.minute, t_pd.second)
Time=[]
for n in range(ds_ovels.time.shape[0]-1):
    results.append(forward_model(t, Lats, Lons, ds_ovels['u'], ds_ovels['v']))
    t += dt
    Time.append(t)
    Lats, Lons = results[-1]['latitude'].values, results[n]['longitude'].values


# create an xarray dataset with the data


In [ ]:
pds = [results[n].drop(columns=['time']) for n in range(len(results))]
drifts = xr.concat([pds[n].to_xarray() for n in range(len(pds))], dim='time')
drifts['time']=xr.DataArray(data=Time, dims='time')
drifts = drifts.rename_dims({'index': 'iceberg'})

In [ ]:
drifts

In [ ]:

# Projection centered on South Pole
proj = ccrs.SouthPolarStereo()

fig = plt.figure(figsize=(10, 10))
ax = plt.subplot(1, 1, 1, projection=proj)

# Set extent: South Pole to ~40°S
ax.set_extent([-180, 180, -90, -40], crs=ccrs.PlateCarree())
ax.add_feature(cfeature.LAND, facecolor='black')

# Add coastlines and grid
ax.coastlines(resolution='110m')
ax.gridlines(draw_labels=True)

# Plot each drifter as a trajectory
# for i in drifts.iceberg.data:  # loop over drifters
for i in range(0, 7):
    lons = drifts['longitude'].isel(iceberg=i).data
    lats = drifts['latitude'].isel(iceberg=i).data
    u0, v0 = drifts['u'].isel(iceberg=i).data, drifts['v'].isel(iceberg=i).data

    ax.plot(
        lons,
        lats,
        transform=ccrs.PlateCarree(),
        label=drifters['Iceberg'].values[i],
        linewidth=6
    )
    ax.plot(
        lons[0],
        lats[0],
        transform=ccrs.PlateCarree(),
        marker='s',
        color='k'
    )
    ax.quiver(lons, lats, u0, v0, transform=ccrs.PlateCarree(), color='gray', scale=3, scale_units='width')

plt.title("Drifter Trajectories near Antarctica")
plt.legend(fontsize=15)
plt.show()